In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

In [2]:
def EstimacionBinomial(n=10, p=0.5, Y=[], densidades=[], thetas=[], i=1, q0=1, q1=1, show=0):

  # Simulación n lanzamientos de Bernoulli
  lanzamientos = np.random.binomial(1, p, size=n).tolist()

  # Simulación pérdida aleatoria de datos
  registro_lanzamientos = []
  if q0 + q1 < 2:
    for res in lanzamientos:
      z = np.random.uniform(0, 1)
      if (res==0 and z<=q0) or (res==1 and z<=q1):
          registro_lanzamientos.append(res)
  else:
    registro_lanzamientos = lanzamientos

  # lanzamientos registrados
  y0 = sum(registro_lanzamientos)
  m = len(registro_lanzamientos) #esto debería ser n (?)

  # Parámetros de la distribución a posteriori
  y = y0
  for y_j in Y:
    y = y + y_j
  a = y + 1
  b = (i+1)*n - y + 1

  # Posibles valores de theta
  theta = np.linspace(0, 1, 1000)

  # Densidad de la distribución a posteriori
  posterior = beta.pdf(theta, a, b)

  # Estimación theta
  hat_theta = a / (a + b)

  if show==1:
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Left: two bars at x=0 and x=1
    axs[0].bar([0, 1], [(m - y0)/m, y0/m], color='C0')
    axs[0].axhline((p*q1)/((1-p)*q0+p*q1), color='C1', linestyle='--', label=f'E(1)={(p*q1)/((1-p)*q0+p*q1)}')
    axs[0].set_xticks([0, 1])
    axs[0].set_xticklabels(['0', '1'])
    axs[0].set_xlabel('Resultado')
    axs[0].set_ylabel('Frecuencia (%)')
    axs[0].set_title(f'Muestra {m}/{n} lanzamientos observados')
    axs[0].set_ylim(0,1)
    axs[0].legend()
    axs[0].grid(axis='y', linestyle='--', alpha=0.7)

    # Right: posterior density and vertical line for estimate
    axs[1].plot(theta, posterior, color='C0', label=f'μ: {hat_theta:.4f}')
    axs[1].axvline(hat_theta, color='C1', linestyle='--', label=f'E(θ): {hat_theta:.2f}')
    axs[1].set_title('Distribución a posteriori')
    axs[1].set_xlabel('θ')
    axs[1].set_ylabel('Densidad')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

  Y.append(sum(lanzamientos))
  densidades.append(posterior)
  thetas.append(hat_theta)

  return hat_theta, i+1, Y, densidades, thetas

In [3]:
def EjemploBinomial(n=10, p=0.5, i_max=3, tol=0.01, q0=1, q1=1, leyenda=0, show=0):

  i = 0
  hat_theta=p
  Y = []
  densidades = []
  thetas = []

  while i < i_max:
    ref = hat_theta
    hat_theta, i, Y, densidades, thetas = EstimacionBinomial(n, p, Y, densidades, thetas, i, q0, q1)

    if abs(ref-hat_theta)<tol:
      break

  if show == 1:
      theta = np.linspace(0, 1, 1000)
      fig, axs = plt.subplots(1, 2, figsize=(14, 5))

      # Cambio la posición de la leyenda a donde menos moleste
      posicion_leyenda = 'upper left' if p > 0.5 else 'upper right'
      print(posicion_leyenda, p)

      # Panel izquierdo: distribuciones a posteriori
      for j, (posterior, theta_hat) in enumerate(zip(densidades, thetas)):
          axs[0].plot(theta, posterior, linewidth=0.7, label=f'Itr {j+1}: E(θ)={theta_hat:.4f}')
      axs[0].set_title('Evolución de las distribuciones a posteriori')
      axs[0].set_xlabel('θ')
      axs[0].set_ylabel('Densidad')
      if leyenda==1:
        axs[0].legend(loc=posicion_leyenda)
      axs[0].grid(True)

      # Panel derecho: evolución de las estimaciones
      axs[1].scatter(range(len(thetas)), thetas, color='C0')
      axs[1].axhline(p, color='C1', linestyle='-', label=f'Valor real: {p}')
      axs[1].axhline(hat_theta, color='C1', linestyle='--', label=f'Estimación: {hat_theta:.4f}')
      axs[1].set_ylim(0, 1)
      axs[1].set_title('Convergencia de la estimación E(θ)')
      axs[1].set_xlabel('Iteración')
      axs[1].set_ylabel('Estimación E(θ)')
      axs[1].legend(loc='lower right')
      axs[1].grid(True)

      plt.tight_layout()
      plt.show()

  return hat_theta, thetas

In [ ]:
hat_theta, thetas = EjemploBinomial(n=10, p=0.7, i_max=15, tol=0.001, q0=1, q1=1, leyenda=1, show=1)

In [4]:
def GibbsSampler(lanzamientos, m, a, b):
  for pos in range(m):
    theta = np.random.beta(a, b)
    z = np.random.uniform(0, 1)
    if z<=theta:
      lanzamientos.append(1)
    else:
      lanzamientos.append(0)
  return lanzamientos

In [5]:
def EstimacionBinomialGibbs(n=10, p=0.5, Y=[], densidades=[], thetas=[], a=1, b=1, i=1, q0=1, q1=1, show=0):

  lanzamientos = np.random.binomial(1, p, size=n).tolist()

  if q0 + q1 < 2:
    registro_lanzamientos = []
    for res in lanzamientos:
      z = np.random.uniform(0, 1)
      if (res==0 and z<=q0) or (res==1 and z<=q1):
          registro_lanzamientos.append(res)
    m = len(registro_lanzamientos)
    lanzamientos = GibbsSampler(registro_lanzamientos, n-m, a, b)

  else: m=n

  #print(registro_lanzamientos)
  y0 = sum(lanzamientos)

  # Parámetros de la distribución a posteriori
  y = y0
  for y_j in Y:
    y = y + y_j
  a = y + 1
  b = (i+1)*n - y + 1

  # Posibles valores de theta
  theta = np.linspace(0, 1, 1000)

  # Densidad de la distribución a posteriori
  posterior = beta.pdf(theta, a, b)

  # Estimación theta
  hat_theta = a / (a + b)

  if show==1:

    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Left: two bars at x=0 and x=1
    axs[0].bar([0, 1], [(n - y0)/n, y0/n], color='C0')
    axs[0].axhline(p, color='C1', linestyle=':', label=f'p = {p}')
    axs[0].axhline((p*q1)/((1-p)*q0+p*q1), color='C1', linestyle='--', label=f'E(1) = {(p*q1)/((1-p)*q0+p*q1)}')
    axs[0].set_xticks([0, 1])
    axs[0].set_xticklabels(['0', '1'])
    axs[0].set_xlabel('Resultado')
    axs[0].set_ylabel('Frecuencia (%)')
    axs[0].set_title(f'Muestra {n} lanzamientos')
    axs[0].set_ylim(0,1)
    axs[0].legend()
    axs[0].grid(axis='y', linestyle='--', alpha=0.7)

    # Right: posterior density and vertical line for estimate
    axs[1].plot(theta, posterior, color='C0', label=f'μ: {hat_theta:.4f}')
    axs[1].axvline(hat_theta, color='C0', linestyle='--', label=f'E(θ): {hat_theta:.2f}')
    axs[1].axvline((p*q1)/((1-p)*q0+p*q1), color='C1', linestyle=':', label=f'E(1)={(p*q1)/((1-p)*q0+p*q1)}')
    axs[1].axvline(p, color='C1', linestyle='-', label=f'p={p}')
    axs[1].set_title('Distribución a posteriori')
    axs[1].set_xlabel('θ')
    axs[1].set_ylabel('Densidad')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

  Y.append(sum(lanzamientos))
  densidades.append(posterior)
  thetas.append(hat_theta)

  return hat_theta, i+1, Y, densidades, thetas, a, b

In [6]:
def EjemploBinomialGibbs(n=10, p=0.5, i_max=3, tol=0.01, q0=1, q1=1, leyenda=0, show=0):

  i = 0
  hat_theta=p
  Y = []
  densidades = []
  thetas = []
  a=1
  b=1

  while i < i_max:
    ref = hat_theta
    hat_theta, i, Y, densidades, thetas, a, b = EstimacionBinomialGibbs(n, p, Y, densidades, thetas, a, b, i, q0, q1)

    if abs(ref-hat_theta)<tol:
      break

  if show == 1:
      theta = np.linspace(0, 1, 1000)
      fig, axs = plt.subplots(1, 2, figsize=(14, 5))

      # Cambio la posición de la leyenda a donde menos moleste
      posicion_leyenda = 'upper left' if p > 0.5 else 'upper right'
      print(posicion_leyenda, p)

      # Panel izquierdo: distribuciones a posteriori
      for j, (posterior, theta_hat) in enumerate(zip(densidades, thetas)):
          axs[0].plot(theta, posterior, linewidth=0.7, label=f'Itr {j+1}: E(θ)={theta_hat:.4f}')
      axs[0].set_title('Evolución de las distribuciones a posteriori')
      axs[0].set_xlabel('θ')
      axs[0].set_ylabel('Densidad')
      if leyenda==1:
        axs[0].legend(loc=posicion_leyenda)
      axs[0].grid(True)

      # Panel derecho: evolución de las estimaciones
      axs[1].scatter(range(len(thetas)), thetas, color='C0')
      axs[1].axhline(p, color='C1', linestyle='-', label=f'Valor real: {p}')
      axs[1].axhline(hat_theta, color='C1', linestyle='--', label=f'Estimación: {hat_theta:.4f}')
      axs[1].set_ylim(0, 1)
      axs[1].set_title('Convergencia de la estimación E(θ)')
      axs[1].set_xlabel('Iteración')
      axs[1].set_ylabel('Estimación E(θ)')
      axs[1].legend(loc='lower right')
      axs[1].grid(True)

      plt.tight_layout()
      plt.show()

  return hat_theta, thetas

In [ ]:
hat_theta, thetas = EjemploBinomialGibbs(n=10, p=0.7, i_max=15, tol=0.001, q0=1, q1=1, leyenda=1, show=1)